This notebook will import helper modelues for modeling and train speech-to-text models.


In [1]:
import os
import sys
sys.path.append("../")
# Custom scripts
from scripts.generate_amharic_characters import GenerateCharacters
from scripts.metadata_loader import MetaDataLoader
from scripts.audio_generator import make_audio_gen
from scripts.model_arch import model_1, train

char_gen = GenerateCharacters()
md_loader = MetaDataLoader()

2022-06-09 21:18:54.280190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-09 21:18:54.280229: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
amharic_root = "../data/AMHARIC"

In [3]:
train_corpus_path = f"{amharic_root}/transcriptions_amharic.csv"
characters = char_gen.get_characters(md_path=train_corpus_path)
characters = sorted(characters)
characters = characters[1:]

In [4]:
char_map = {}
char_map[""] = 0
char_map["<SPACE>"] = 1
index = 2
for c in characters:
    char_map[c] = index
    index += 1
index_map = {v+1: k for k, v in char_map.items()}

In [5]:
TRAIN_CORPUS = os.path.join(amharic_root, "train_corpus.json")
VALID_CORPUS = os.path.join(amharic_root, "valid_corpus.json")

MFCC_DIM = 13
SPECTOGRAM = True
EPOCHS = 1
MODEL_NAME = "RNN_model"


MINI_BATCH_SIZE = 64

SORT_BY_DURATION=False
MAX_DURATION = 10.0

audio_gen = make_audio_gen(TRAIN_CORPUS, 
                        VALID_CORPUS, 
                        spectrogram=False, 
                        mfcc_dim=MFCC_DIM,
                        minibatch_size=MINI_BATCH_SIZE, 
                        sort_by_duration=SORT_BY_DURATION,
                        max_duration=MAX_DURATION, 
                        char_map=char_map)
# add the training data to the generator
audio_gen.load_train_data()
audio_gen.load_validation_data()

8700
2175


In [6]:
model = model_1(input_dim=MFCC_DIM,
                units=5,
                activation='relu',
                output_dim=len(char_map)+1,
                )

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 13)]        0         
                                                                 
 rnn (GRU)                   (None, None, 5)           300       
                                                                 
 batch_normalization (BatchN  (None, None, 5)          20        
 ormalization)                                                   
                                                                 
 time_distributed (TimeDistr  (None, None, 224)        1344      
 ibuted)                                                         
                                                                 
 softmax (Activation)        (None, None, 224)         0         
                                                                 
Total params: 1,664
Trainable params: 1,654
Non-trainable par

2022-06-09 21:19:00.918004: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-09 21:19:00.918054: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-60-46.ec2.internal): /proc/driver/nvidia/version does not exist
2022-06-09 21:19:00.918739: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:

train(audio_gen, 
    input_to_softmax=model, 
    model_name=MODEL_NAME, 
    epochs=EPOCHS, 
    minibatch_size=MINI_BATCH_SIZE
    )

../scripts/model_arch.py:93: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  callbacks=[checkpointer], verbose=verbose, use_multiprocessing=True)


114/114 [==============================] - 147s 1s/step - loss: nan - val_loss: nan
{'loss': [nan], 'val_loss': [nan]}
